In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

这本笔记本是由[Jesus Chavez](https://github.com/jchavezar)编写的笔记本的更新版本。

在Google云平台上进行端到端机器学习：MLOps阶段2：实验：使用DASK开始分布式训练

在Colab中运行

在GitHub上查看

在Vertex AI Workbench中打开

## 概述

本教程演示了如何使用`Vertex AI Training`为XGBoost模型进行分布式训练。本教程包括了使用DASK支持XGBoost和Scikit-learn模型的分布式训练，以及使用Flask作为自定义服务容器的Web服务器。

### 目标

在本教程中，您将学习如何使用`Vertex AI Training`来进行XGBoost模型的分布式训练，使用OSS包DASK。此外，您还将学习如何使用Flask Web服务器构建和部署自定义Serving容器。

本教程使用以下谷歌云机器学习服务和资源：

- `Vertex AI Training`
- `Vertex AI Prediction`
- `Vertex AI Model` 资源
- `Vertex AI Endpoint` 资源

执行的步骤包括：

- 使用DASK构建XGBoost训练脚本进行分布式训练。
- 构建自定义的训练容器。
- 配置分布式自定义训练作业。
- 执行自定义训练作业。
- 使用Flask构建自定义Serving容器。
- 将训练好的XGBoost模型上传为`Vertex AI Model`资源。
- 创建一个`Vertex AI Endpoint`资源。
- 将`Vertex AI Model`资源部署到`Vertex AI Endpoint`资源。
- 进行预测。

数据集

本教程使用的数据集是[森林覆盖类型](https://archive.ics.uci.edu/ml/datasets/covertype)。该数据集的版本以 CSV 格式存储在一个公共云存储桶中。该数据集仅从地图变量预测森林覆盖类型。

###  费用

本教程使用 Google Cloud 的可计费组件:

* Vertex AI
* 云存储

了解 [Vertex AI 的定价](https://cloud.google.com/vertex-ai/pricing) 和 [云存储的定价](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用情况生成费用估算。

## 安装

安装以下软件包以执行此笔记本。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG -q

### 重新启动内核

在安装了额外的软件包之后，您需要重新启动笔记本内核，这样它才能找到这些软件包。

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### 设置您的Google Cloud项目

**无论您使用什么笔记本环境，都需要按照以下步骤操作。**

1. [选择或创建Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。您首次创建账户时，可获得300美元的免费信贷，可用于计算/存储成本。

2. [确保您的项目已启用计费功能。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用以下API: Vertex AI API、Compute Engine API和Cloud Storage。](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component,storage-component.googleapis.com)

4. 如果您在本地运行此笔记本，则需要安装[Cloud SDK]((https://cloud.google.com/sdk))。

5. 在下面的单元格中输入您的项目ID。然后运行该单元格，确保Cloud SDK在本笔记本的所有命令中使用正确的项目。

**注意**：Jupyter运行以`!`为前缀的行作为shell命令，并插入以`$`为前缀的Python变量。

设定您的项目ID

**如果您不知道您的项目ID**，您可以使用`gcloud`来获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

区域

您还可以更改“REGION”变量，该变量用于本笔记本其余部分的操作。以下是Vertex AI支持的区域。我们建议您选择离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太：`asia-east1`

您可能无法在Vertex AI中使用多区域存储桶进行训练。并非所有区域都为所有Vertex AI服务提供支持。

了解更多关于[Vertex AI地区](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

时间戳

如果您正在进行直播教程会话，您可能正在使用一个共享的测试账户或项目。为避免在创建的资源上发生名称冲突，您应该为每个实例会话创建一个时间戳，并将时间戳附加到您在本教程中创建的资源名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 认证您的Google Cloud帐户

**如果您正在使用Vertex AI Workbench笔记本**，您的环境已经通过身份验证。跳过此步骤。

**如果您正在使用Colab**，请运行下面的单元格，并在提示时按照说明通过oAuth认证您的帐户。

**否则**，请按照以下步骤操作：

在Cloud控制台中，转到[创建服务帐户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击Create service account**。

在**服务帐户名称**字段中输入一个名称，并点击**创建**。

在**授予该服务帐户对项目的访问权限**部分，点击角色下拉列表。在过滤框中键入“Vertex”，并选择**Vertex管理员**。在过滤框中键入“Storage Object Admin”，并选择**Storage Object Admin**。

点击创建。一个包含您密钥的JSON文件将下载到您的本地环境中。

将您的服务帐户密钥路径输入为下面单元格中的GOOGLE_APPLICATION_CREDENTIALS变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

创建云存储桶

**无论您使用哪种笔记本环境，都需要以下步骤。**

当您初始化用于Python的Vertex AI SDK时，需要指定一个云存储暂存桶。暂存桶是您的数据集和模型资源相关数据在会话中保留的地方。

在下方设置您的云存储桶名称。存储桶名称必须在所有谷歌云项目中全局唯一，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP
    BUCKET_URI = "gs://" + BUCKET_NAME

只有当您的存储桶尚不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过检查Cloud Storage桶的内容来验证访问权限。

In [ ]:
! gsutil ls -al $BUCKET_URI

### 启用Artifact Registry API

您必须为您的项目启用Artifact Registry API服务。

了解更多关于[启用服务](https://cloud.google.com/artifact-registry/docs/enable-service)。

In [ ]:
! gcloud services enable artifactregistry.googleapis.com

### 设置变量

接下来，设置一些在本教程中使用的变量。
### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aiplatform

### 初始化用于 Python 的 Vertex AI SDK

为您的项目和相应的存储桶初始化用于 Python 的 Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

## DASK简介

摘自XGBoost DASK文档。

### 什么是DASK?

```
Dask是一个基于Python的并行计算库。Dask允许轻松管理分布式工作节点，并擅长处理大规模的分布式数据科学工作流。XGBoost中的实现源自dask-xgboost，具有一些扩展功能和不同的接口。
```

### XGBoost DASK概述

```
一个dask集群由三个不同组件组成：一个集中式调度器，一个或多个工作节点，以及一个或多个充当用户操作入口的客户端。当与dask一起使用XGBoost时，需要从客户端调用XGBoost dask接口。以下是一个简单的示例，演示在dask集群上运行XGBoost的基本用法。
```

了解更多关于[XGBoost DASK](https://xgboost.readthedocs.io/en/stable/tutorials/dask.html)

了解更多关于[DASK](dask.org)

## XGBoost培训介绍

一旦您训练了一个XGBoost模型，您将希望将其保存在云存储位置，以便随后上传到`Vertex AI Model`资源。
XGBoost软件包没有支持将模型保存到云存储位置。相反，您将执行以下步骤将其保存到云存储位置。

1. 将内存中的模型保存到本地文件系统（例如，model.bst）。
2. 使用`google.cloud.storage`将本地副本复制到指定的云存储位置。

### 检查培训包

#### 包布局

在开始培训之前，您将查看如何组装用于自定义训练作业的Python包。解压缩后，该包包含以下目录/文件布局。

- PKG-INFO
- README.md
- trainer
  - \_\_init\_\_.py
  - task.py

文件`trainer/task.py`是用于执行自定义训练作业的Python脚本。*注意*，在工作池规范中引用它时，您将目录斜杠替换为点号（`trainer.task`），并删除文件后缀（`.py`）。

#### 包装配

在下面的单元格中，您将组装培训包。

In [ ]:
# Make folder for Python training script
! rm -rf custom prediction
! mkdir custom prediction

# Add package information
! touch custom/README.md

pkg_info = "Metadata-Version: 1.0\n\nName: cover_type classification\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

## 使用Dask + CUDA（GPU）构建训练脚本

接下来，您将编写自定义训练脚本，使用DASK进行分布式训练来训练XGBoost模型。

- `args`：传递给训练脚本的参数：
  - `dataset-source`：包含训练数据的CSV文件的云存储位置。
  - `model_dir`：用于存储模型构件的云存储位置。
  - `model_name`：模型的文件名。
  - `--num-gpu-per-worker`：对于调度程序，每个worker的GPU数量。您还需要在`run()`方法中进一步设置分配给作业的相同数量的GPU的`accelerator_count`。
  - `--threads-per-worker`：为了提高效率，您应该将每个worker的线程数设置为GPU的数量。
- `get_scheduler_info()`：获取用于设置分布式集群的VM/进程调度相关信息。
- `using_quantile_device_dmatrix()`：执行分布式训练：
  - 注意`client`是用于分布式训练的集群控制器。
  - 使用`dask_cudf.read_csv()`从CSV文件中读取数据集。
  - 将数据集拆分并预处理为训练/评估集。
  - 使用`dxgb.DaskDeviceQuantileDMatrix()`加载用于分布式训练的数据集。
  - 使用`xgb.dask.train()`进行分布式训练。
- `saved_model()`：将模型和评估指标保存到指定的云存储位置。

In [ ]:
%%writefile custom/trainer/task.py

import argparse
import os
import logging
import dask_cudf
import xgboost as xgb
import pandas as pd
import subprocess
import time
from google.cloud import storage

from dask_cuda import LocalCUDACluster
from dask.distributed import Client
from dask.distributed import wait
from dask import array as da
from xgboost import dask as dxgb
from xgboost.dask import DaskDMatrix
from dask.utils import parse_bytes

parser = argparse.ArgumentParser()
parser.add_argument(
    '--dataset-source', dest='dataset',
    type=str,
    help='Dataset.')
parser.add_argument(
    '--model-dir',
    default=os.getenv('AIP_MODEL_DIR'),
    help='GCS location to export models')
parser.add_argument(
    '--model-name',
    default="custom-train",
    help='The name of your saved model')
parser.add_argument(
    '--num-gpu-per-worker', type=str, help='num of workers',
    default=2)
parser.add_argument(
    '--threads-per-worker', type=str, help='num of threads per worker',
    default=4)

args = parser.parse_args()


def save_model(model_dir):
    logging.info(f"Reading input job_dir: {model_dir}")
    model_dir = model_dir.split("/")
    bucket_name = model_dir[2]
    object_prefix = "/".join(model_dir[3:]).rstrip("/")
    logging.info(f"Reading object_prefix: {object_prefix}")

    if object_prefix:
        model_path = '{}/{}'.format(object_prefix, "xgboost")
    else:
        model_path = '{}'.format("xgboost")
            
    logging.info(f"The model path is {model_path}")
    bucket = storage.Client().bucket(bucket_name)    
    local_path = os.path.join("/tmp", "xgboost")
    
    files = [f for f in os.listdir(local_path) if os.path.isfile(os.path.join(local_path, f))]
    for file in files:
        local_file = os.path.join(local_path, file)
        blob = bucket.blob("/".join([model_path, file]))
        blob.upload_from_filename(local_file)
    logging.info(local_file)
    logging.info(f"gs://{bucket_name}/{model_path}")
    logging.info(f"Saved model files in gs://{bucket_name}/{model_path}")

        
def using_quantile_device_dmatrix(client: Client, 
                                  dataset_source: str, 
                                  model_dir: str, 
                                  model_name: str):
    
    start_time = time.time()
    logging.info(f"Importing dataset {dataset_source}")
    df = dask_cudf.read_csv(dataset_source)

    logging.info("Cleaning and standarizing dataset")
    df = df.dropna() 

    logging.info("Splitting dataset")
    df_train, df_eval = df.random_split([0.8, 0.2], random_state=123)
    df_train_features= df_train.drop('Cover_Type', axis=1)
    df_eval_features= df_eval.drop('Cover_Type', axis=1)
    df_train_labels = df_train.pop('Cover_Type')
    df_eval_labels = df_eval.pop('Cover_Type')

    logging.info("Train Dataset for dask")
    dtrain = dxgb.DaskDeviceQuantileDMatrix(client, df_train_features, df_train_labels)
    
    logging.info("Eval Dataset for dask")
    dvalid = dxgb.DaskDeviceQuantileDMatrix(client, df_eval_features, df_eval_labels)
    logging.info("[INFO]: ------ QuantileDMatrix is formed in {} seconds ---".format((time.time() - start_time)))

    del df_train_features
    del df_train_labels
    del df_eval_features
    del df_eval_labels 
    
    start_time = time.time()
    
    logging.info("Training")
    logging.info(f"XGBoost version: {xgb.__version__}")
    output = xgb.dask.train(
        client,
        {
            "verbosity": 2, 
            "tree_method": "gpu_hist", 
            "objective": "multi:softprob",
            "eval_metric": ["mlogloss"],
            "learning_rate": 0.1,
            "gamma": 0.9,
            "subsample": 0.5,
            "max_depth": 9,
            "num_class": 8
        },
        dtrain,
        num_boost_round=10,
        evals=[(dvalid, "valid1")],
        early_stopping_rounds=5
    ) 
    print("[INFO]: ------ Training is completed in {} seconds ---".format((time.time() - start_time)))

    # Saving models and exporting performance metrics
    
    df_eval_metrics = pd.DataFrame(output["history"]["valid1"])
    model = output["booster"]
    best_model = model[: model.best_iteration]
    print(f"Best model: {best_model}")
    
    temp_dir = "/tmp/xgboost"
    os.mkdir(temp_dir)
    best_model.save_model("{}/{}".format(temp_dir, model_name))
    df_eval_metrics.to_json("{}/all_results.json".format(temp_dir))

    save_model(model_dir)
        
def get_scheduler_info():
    scheduler_ip =  subprocess.check_output(['hostname','--all-ip-addresses'])
    scheduler_ip = scheduler_ip.decode('UTF-8').split()[0]
    scheduler_port = '8786'
    scheduler_uri = '{}:{}'.format(scheduler_ip, scheduler_port)
    return scheduler_ip, scheduler_uri

if __name__ == '__main__':
    print("Creating dask cluster")
    
    sched_ip, sched_uri = get_scheduler_info()
    
    print(f"Sched_ip and Sched_uri, {sched_ip}, {sched_uri}")

    print("[INFO]: ------ LocalCUDACluster is being formed ")
    
    with LocalCUDACluster(
        ip=sched_ip,
        n_workers=int(args.num_gpu_per_worker), 
        threads_per_worker=int(args.threads_per_worker) 
    ) as cluster:
        with Client(cluster) as client:
            print('[INFO]: ------ Calling main function ')
            using_quantile_device_dmatrix(client, 
                                          dataset_source=args.dataset, 
                                          model_dir=args.model_dir, 
                                          model_name=args.model_name
                                         )

### 构建自定义训练容器

接下来，您将构建一个支持GPU CUDA的用于训练XGBoost模型的自定义（Docker）容器。作为基础镜像，您将使用带有CUDA支持的Rapids AI镜像。然后，您将在该镜像中安装XGBoost、Google Cloud Fuse和Cloud Storage的软件包。

*备注：*目前，Vertex AI不支持具有GPU支持的XGBoost的预构建容器（仅支持CPU）。

In [ ]:
%%writefile custom/Dockerfile

FROM rapidsai/rapidsai-nightly:22.04-cuda11.2-base-ubuntu20.04-py3.9

RUN pip install google.cloud[storage] \
  && pip install gcsfs \
  && pip install xgboost --upgrade

COPY trainer trainer/

ENTRYPOINT ["python", "trainer/task.py"]

## 创建一个私有的Docker仓库

接下来，在Google Artifact Registry中创建您自己的Docker仓库。

1. 运行`gcloud artifacts repositories create`命令，使用您的区域创建一个新的Docker仓库，描述为"docker仓库"。

2. 运行`gcloud artifacts repositories list`命令，验证您的仓库是否已经创建。

In [ ]:
PRIVATE_REPO = "my-docker-repo"

! gcloud artifacts repositories create {PRIVATE_REPO} --repository-format=docker --location={REGION} --description="Docker repository"

! gcloud artifacts repositories list

### 配置私有存储库的验证

在推送或拉取容器图像之前，配置Docker使用`gcloud`命令行工具对您所在地区的`Artifact Registry`进行验证请求。

In [ ]:
! gcloud auth configure-docker {REGION}-docker.pkg.dev --quiet

接下来，您将构建自定义培训（Docker）映像。

In [ ]:
TRAIN_IMAGE = (
    f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{PRIVATE_REPO}/train_gpu_xgb:latest"
)

! docker build custom -t $TRAIN_IMAGE
! docker push $TRAIN_IMAGE

## 使用Flask构建serving容器。

在本教程中，将使用自定义的serving容器来提供模型。您可以使用Flask构建用于健康和预测路由的HTTP服务器。

*TODO: 解除项目ID的硬编码*  

In [ ]:
%%writefile prediction/app.py

import os
import logging
import pandas as pd
import xgboost as xgb
from flask import Flask, request, Response, jsonify
from google.cloud import storage

#client = storage.Client(project=os.environ['PROJECT_ID'])
client = storage.Client(project='andy-1234-221921')

# Model Download from gcs

fname = "model.json"

with open(fname, "wb") as model:
    client.download_blob_to_file(
        f"{os.environ['AIP_STORAGE_URI']}/{fname}", model
    )

# Loading model
print("Loading model from: {}".format(fname))
model = xgb.Booster(model_file=fname)

# Creation of the Flask app
app = Flask(__name__)

# Flask route for Liveness checks
@app.route(os.environ['AIP_HEALTH_ROUTE'])
def isalive():
    status_code = Response(status=200)
    return status_code

# Flask route for predictions
@app.route(os.environ['AIP_PREDICT_ROUTE'],methods=['GET','POST'])
def prediction():
    _features = ['Id','Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
                          'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm','Horizontal_Distance_To_Fire_Points', 'Wilderness_Area1', 'Wilderness_Area2', 'Wilderness_Area3', 
                          'Wilderness_Area4', 'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5', 'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9',
                          'Soil_Type10','Soil_Type11','Soil_Type12','Soil_Type13','Soil_Type14','Soil_Type15','Soil_Type16','Soil_Type17','Soil_Type18','Soil_Type19', 
                          'Soil_Type20', 'Soil_Type21', 'Soil_Type22', 'Soil_Type23', 'Soil_Type24', 'Soil_Type25', 'Soil_Type26', 'Soil_Type27', 'Soil_Type28', 'Soil_Type29',
                          'Soil_Type30', 'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34', 'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 'Soil_Type38', 'Soil_Type39', 'Soil_Type40']
    data = request.get_json(silent=True, force=True)
    dmf = xgb.DMatrix(pd.DataFrame(data["instances"], columns=_features))
    response = pd.DataFrame(model.predict(dmf))
    logging.info(f"Response: {response}")
    return jsonify({"Cover Type": str(response.idxmax(axis=1)[0])})

if __name__ == "__main__":
    app.run(debug=True, host='0.0.0.0', port=8080)

### 为您定制的容器构建软件包要求

接下来，您需要为您的定制容器构建软件包要求文件。

In [ ]:
%%writefile prediction/requirements.txt

google-cloud-storage
numpy
pandas
flask
xgboost

接下来，您需构建一个定制的（Docker）容器，用于从您部署的XGBoost模型提供预测。

In [ ]:
%%writefile prediction/Dockerfile

FROM python:3.7-buster

RUN mkdir my-model

COPY app.py ./app.py
COPY requirements.txt ./requirements.txt
RUN pip install -r requirements.txt 

# Flask Env Variable
ENV FLASK_APP=app

# Expose port 8080
EXPOSE 8080

CMD flask run --host=0.0.0.0 --port=8080

自定义服务容器

接下来，您将构建您的自定义预测（Docker）镜像。

In [ ]:
DEPLOY_IMAGE = (
    f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{PRIVATE_REPO}/predict_gpu_xgb:latest"
)

! docker build prediction -t $DEPLOY_IMAGE
! docker push $DEPLOY_IMAGE

将培训脚本存储到您的云存储桶中

接下来，您将培训文件夹打包成压缩的tar文件，并将其存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_URI/trainer_covertype.tar.gz

### 创建并运行自定义训练任务

要训练一个自定义模型，您需要执行两个步骤：1）创建一个自定义训练任务，2）运行这个任务。

#### 创建自定义训练任务

使用`CustomTrainingJob`类创建一个自定义训练任务，具有以下参数：

- `display_name`：自定义训练任务的可读名称。
- `container_uri`：训练容器镜像。

- `command`：容器中要调用的命令（如解释器）和脚本。

- `model_serving_container_image_uri`：在模型部署时要与之一起使用的对应的服务容器。

*注意：* 容器内的解释器和要调用的脚本可以在容器中被覆盖（即ENTRYPOINT）。

In [ ]:
DISPLAY_NAME = "covertype_" + TIMESTAMP

job = aiplatform.CustomContainerTrainingJob(
    display_name="DISPLAY_NAME",
    container_uri=TRAIN_IMAGE,
    command=["python3", "trainer/task.py"],
    model_serving_container_image_uri=DEPLOY_IMAGE,
)

#### 运行自定义容器训练任务

接下来，您通过调用`run()`方法来运行自定义作业，以启动训练任务，传入以下参数：

- `model_display_name`：当训练任务完成时，模型工件将自动上传为指定显示名称的`Vertex AI模型`资源。
- `args`：要传递给训练脚本的命令行参数：
    - `dataset-source`：CSV数据集文件的Cloud Storage位置。
    - `model-name`：模型工件的文件名。
    - `num-gpu-per-worker`：每个VM实例（worker）的GPU数量。
    - `threads-per-worker`：每个VM实例的训练进程线程数。
- `replica_count`：VM实例的数量。
- `machine_type`：每个VM实例的机器类型。
- `accelerator_type`：GPU的类型，如果有的话。
- `accelerator_count`：每个VM实例的GPU数量，如果有的话。
- `base_output_dir`：保存模型工件的Cloud Storage位置。

In [ ]:
DATASET_FILE = "gs://vtx-datasets-public/cover_type_4Mrows.csv"

MODEL_DIR = f"{BUCKET_URI}"

NGPU = 4

CMDARGS = [
    "--dataset-source",
    DATASET_FILE,
    "--model-name",
    "model.json",
    "--num-gpu-per-worker",
    str(NGPU),
    "--threads-per-worker",
    "4",
]

model = job.run(
    model_display_name="covertype_" + TIMESTAMP,
    args=CMDARGS,
    replica_count=1,
    machine_type="n1-standard-4",
    accelerator_type="NVIDIA_TESLA_T4",
    accelerator_count=NGPU,
    base_output_dir=MODEL_DIR,
)

删除自定义训练作业

在训练作业完成后，您可以使用 `delete()` 方法删除训练作业。在完成之前，训练作业可以使用 `cancel()` 方法取消。

In [ ]:
job.delete()

显示已保存模型工件的位置

接下来，您可以显示云存储位置的内容，您的训练脚本保存了训练模型的工件。

In [ ]:
! gsutil ls {MODEL_DIR}/model

### 部署模型

接下来，您可以使用`deploy()`方法将您的XGBoost模型+服务容器部署到`Vertex AI Endpoint`资源。

In [ ]:
endpoint = model.deploy(machine_type="n1-standard-4")

准备预测请求

接下来，您准备一个预测请求。为了演示目的，您使用数据集中的第一行（示例）。

In [ ]:
output = ! gsutil cat {DATASET_FILE} | head -n2

print(output[1])

import json

instance = json.loads(output[1])
print(instance)

### 进行预测

BLAH

In [ ]:
prediction = endpoint.predict(instances=[instance])
print(prediction)

解除部署并删除“端点”资源

In [ ]:
endpoint.undeploy_all(force=True)
endpoint.delete()

#### 删除 `Model` 资源

您可以使用 delete() 方法删除一个模型资源。

In [ ]:
model.delete()

清理

In [ ]:
delete_bucket = True
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r {BUCKET_URI}

! rm -rf custom prediction custom.tar.gz

# TODO: delete repo, delete images